In [1]:
import json
import os
from glob import glob
from tqdm import tqdm
from collections import defaultdict
from transformers import BertTokenizer
import pickle
import argparse

2021-11-16 15:32:51.379827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-16 15:32:51.379894: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
indir = os.path.join('data','preprocessed')
outdir = os.path.join('data','preprocessed')
files = sorted(glob(os.path.join(indir, '*_tokenized.jsonlist')))
word_file = 'constitution_words.txt'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
# tokenize key words and phrases
with open(word_file, 'r',encoding = 'utf-8') as f:
    target_words = f.read().splitlines()
target_words = [tokenizer.tokenize(x) for x in target_words]
target_words_cleaned = []
for x in target_words:
    # rejoin into concatenated words
    rejoined_pieces = []
    for p_i, piece in enumerate(x):
        if p_i == 0:
            rejoined_pieces.append(piece)
        elif piece.startswith('##'):
            rejoined_pieces[-1] += piece
        else:
            rejoined_pieces.append(piece)
    target_words_cleaned.append(rejoined_pieces)

In [4]:
index = defaultdict(set)
# collect index of tokens in the documents
for i,file in enumerate(files):
    with open(file) as f:
        docs = f.readlines()
    for x,doc in enumerate(tqdm(docs)):
        doc = json.loads(doc)
        tokens = doc['tokens']
        doc_index = 0
        token = []
        token_index = 0
        while doc_index < len(tokens):
            if doc_index == 0:
                token = tokens[doc_index]
            elif tokens[doc_index].startswith('##'):
                token += tokens[doc_index]    
            else:
                #save the current token and its start index
                index[token].add((i,x,token_index))
                #start a new token
                token = tokens[doc_index]
                token_index = doc_index
            #save the final token if we are at the end   
            if doc_index == len(tokens)-1:
                index[token].add((i,x,token_index))
            
            #update the index based on the broken up tokens, not the whole word             
            doc_index += 1 

100%|███████████████████████████████████████| 174686/174686 [13:23<00:00, 217.45it/s]


In [19]:
# additional check
target_index['bear arms']

{(0, 4204, 93),
 (0, 8694, 525),
 (0, 23623, 1272),
 (0, 23651, 231),
 (0, 26167, 5103),
 (0, 26679, 634),
 (0, 26722, 151),
 (0, 26737, 3345),
 (0, 26737, 4268),
 (0, 26784, 141),
 (0, 26859, 900),
 (0, 27970, 7263),
 (0, 30033, 310),
 (0, 30078, 226),
 (0, 30279, 189),
 (0, 30313, 903),
 (0, 34344, 787),
 (0, 36942, 173),
 (0, 37742, 2611),
 (0, 38409, 61),
 (0, 40106, 6708),
 (0, 40109, 687),
 (0, 55185, 1010),
 (0, 55186, 1341),
 (0, 55199, 1066),
 (0, 56105, 892),
 (0, 56698, 37),
 (0, 56700, 42),
 (0, 56787, 233),
 (0, 57288, 1441),
 (0, 57498, 292),
 (0, 57783, 656),
 (0, 57849, 54),
 (0, 57882, 50),
 (0, 57911, 193),
 (0, 58307, 2111),
 (0, 58399, 125),
 (0, 59381, 372),
 (0, 60110, 580),
 (0, 60368, 730),
 (0, 62057, 1145),
 (0, 62059, 924),
 (0, 63295, 760),
 (0, 64055, 84),
 (0, 65555, 292),
 (0, 68149, 638),
 (0, 69131, 46),
 (0, 70288, 2086),
 (0, 73429, 1725),
 (0, 76556, 202),
 (0, 78758, 1974),
 (0, 81542, 1886),
 (0, 81747, 1182),
 (0, 87533, 139),
 (0, 95231, 785),
 (

In [5]:
target_index = {}
for word in tqdm(target_words_cleaned):
    if len(word) == 1:
        # just one word
        target_index[word[0]]=index.get(word[0],set())

    elif len(word) > 1:
        # we have a phrase
        phrase_indexes = []
        start = index.get(word[0])
        for f_id,doc_id,doc_index in start:
            match = False
            for x,piece in enumerate(word[1:]):
                if (f_id,doc_id,doc_index+x+1) in index.get(piece,set()):  
                    match = True
                else:
                    match = False
                    break

            if match:
                phrase_indexes.append((f_id,doc_id,doc_index))
        target_index[' '.join(word)] = set(phrase_indexes)


100%|███████████████████████████████████████████| 1053/1053 [00:01<00:00, 613.24it/s]


In [22]:
with open(os.path.join(outdir,'target_word_index.dict'),'wb') as f:
    pickle.dump(target_index,file=f)